In [1]:
import os
import sys

# Set the working directory to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.config import Config

# Create default config
# DEFAULT_CONFIG_PATH = os.path.join(os.path.dirname(__file__), '../config/config.yaml')
Config = Config.from_yaml(os.path.join(project_root, 'config/config.yaml'))

import numpy as np
import tensorflow as tf
import strawberryfields as sf
from strawberryfields.ops import *
import pickle
import random as rd
import warnings
from datetime import datetime
import uncertainty_toolbox as uct
from itertools import product
from tqdm import tqdm

from src.dataloader import get_data, quartic_data
from src.plotting import plot_predictions, plot_training_results, plot_predictions_new, plot_eval_metrics, plot_mlp_architecture_vs_rmse, plot_data
from src.baseline import train_mlp_baseline, predict_mlp_baseline, train_polynomial_baseline, predict_polynomial_baseline
from src.uq import selective_prediction, compute_eval_metrics
from src.model import train_memristor, predict_memristor, build_circuit
from src.utils import format_metrics, format_hyperparameters
from src.logger import log_experiment_id

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
rd.seed(42)



In [4]:
# Create directory called experiment_CONFIG.LOG_NAME in reports/logs
date = datetime.now().strftime('%Y%m%d_%H%M%S')
os.makedirs(f"reports/baseline_{date}/logs", exist_ok=False)
os.makedirs(f"reports/baseline_{date}/plots", exist_ok=False)

Config.log_file_name = f"reports/baseline_{date}/log.txt"
Config.plot_path = f"reports/baseline_{date}/plots/"
Config.param_id = f"hp_s{Config.training.steps}_lr{Config.training.learning_rate}_md{Config.training.memory_depth}_cd{Config.training.cutoff_dim}"    

# Log experiment id
log_experiment_id(Config.log_file_name, Config.param_id, "MLP Baseline Search")

X_train, y_train, X_test, y_test, _ = get_data(n_data=Config.data.n_data, 
                                                sigma_noise_1=Config.data.sigma_noise_1, 
                                                datafunction=quartic_data
                                                )

# plot_data(X_train, y_train, X_test, y_test, save_path=Config.plot_path + "data.png")


print("Training MLP Baseline Models...")
mlp_architectures = [[20], [20, 20], [20, 20, 20], [20, 20, 20, 20]]
rmse_values = []

for hidden_layers in mlp_architectures:
    mlp_model = train_mlp_baseline(
        X_train, 
        y_train, 
        hidden_layers=hidden_layers,
        epochs=10,
        learning_rate=0.01
    )
    mlp_predictions = predict_mlp_baseline(mlp_model, X_test)
    metrics, metric_categories = compute_eval_metrics(
        np.array(mlp_predictions), 
        np.array(y_test), 
        np.array([]),
        Config.log_file_name,
        Config.param_id
    )

    rmse_values.append(metrics['accuracy']['rmse'])
# plot mlp architecture vs rmse using a new plotting function

import matplotlib.pyplot as plt

# Define the plotting function
def plot_mlp_architecture_vs_rmse(architectures, rmse_values, save_path=None):
    plt.figure(figsize=(10, 6))
    plt.plot([str(arch) for arch in architectures], rmse_values, marker='o')
    plt.xlabel('MLP Architecture (Hidden Layers)')
    plt.ylabel('RMSE')
    plt.title('MLP Architecture vs RMSE')
    plt.grid(True)
    
    if save_path is not None:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()


# Plot MLP architecture vs RMSE
plot_mlp_architecture_vs_rmse(
    mlp_architectures,
    rmse_values,
    save_path=Config.plot_path + "mlp_architecture_vs_rmse.png"
)



Training MLP Baseline Models...


/Users/anani/Code/uq-qnn/.venv/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
 (1/n) Calculating accuracy metrics


/Users/anani/Code/uq-qnn/.venv/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
 (1/n) Calculating accuracy metrics


/Users/anani/Code/uq-qnn/.venv/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
 (1/n) Calculating accuracy metrics


/Users/anani/Code/uq-qnn/.venv/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
 (1/n) Calculating accuracy metrics


Training MLP Baseline Models...


/Users/anani/Code/uq-qnn/.venv/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
 (1/n) Calculating accuracy metrics


/Users/anani/Code/uq-qnn/.venv/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 971us/step
 (1/n) Calculating accuracy metrics


...

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
 (1/n) Calculating accuracy metrics


/Users/anani/Code/uq-qnn/.venv/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 998us/step
 (1/n) Calculating accuracy metrics


/Users/anani/Code/uq-qnn/.venv/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
 (1/n) Calculating accuracy metrics
